In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pyautogui as pag
import time
import pickle
from solver import solve, solve_and_autoclick

In [34]:
def save_words_by_length():
    with open("words.txt") as f:
        words = [line.strip() for line in f.readlines() if len(line.strip()) >= 4]
    max_word_length = max(len(word) for word in words)
    words_by_length = {i: [w for w in words if len(w) == i] for i in range(4, max_word_length + 1)}
    with open("words_by_length.pkl", "wb") as f:
        pickle.dump(words_by_length, f)

In [35]:
save_words_by_length()

In [ ]:
# grid_dict = {(x,y): grid[y, x] for y in range(grid.shape[0]) for x in range(grid.shape[1])}
# grid_dict2 = {k:v for k,v in grid_dict.items() if v != ''}
# fig, ax = plt.subplots()
# nx.draw(graph, labels=grid_dict2, ax=ax)

In [ ]:
#sorted_words = sorted(found_words, key=lambda s: (len(s), s))

In [38]:
    with open("words.txt") as f:
        words = [line.strip() for line in f.readlines() if len(line.strip()) >= 4]

In [42]:
foo = solve("elgspoauhhcracas")

In [43]:
foo[-5:]

['auras', 'sauch', 'sacral', 'halo', 'haole']

In [41]:
solve_and_autoclick("elgspoauhhcracas", delay=4)

In [39]:
"chai" in words

False

amid
